# Summarizing WEB Articles from wikipedia

In [2]:
#pip install beautifulsoup4

In [1]:
#pip install lxml
#pip install nltk

In [2]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

# Récupérer des articles de Wikipedia (SCRAPING)

In [3]:
import bs4 as bs
import urllib.request
import re

scraped_data = urllib.request.urlopen('https://en.wikipedia.org/wiki/Natural_language_processing')
article = scraped_data.read()

parsed_article = bs.BeautifulSoup(article,'lxml')

paragraphs = parsed_article.find_all('p')

article_text = ""

for p in paragraphs:
    article_text += p.text

In [4]:
print (article_text)

Natural language processing (NLP) is a subfield of linguistics, computer science, and artificial intelligence concerned with the interactions between computers and human language, in particular how to program computers to process and analyze large amounts of natural language data.  The goal is a computer capable of "understanding" the contents of documents, including the contextual nuances of the language within them. The technology can then accurately extract information and insights contained in the documents as well as categorize and organize the documents themselves.
Challenges in natural language processing frequently involve speech recognition, natural-language understanding, and natural-language generation.
Natural language processing has its roots in the 1950s. Already in 1950, Alan Turing published an article titled "Computing Machinery and Intelligence" which proposed what is now called the Turing test as a criterion of intelligence, though at the time that was not articulate

In [5]:
len(article_text)

8945

In [6]:
# Removing Square Brackets and Extra Spaces
article_text = re.sub(r'\[[0-9]*\]', ' ', article_text)
article_text = re.sub(r'\s+', ' ', article_text)

In [7]:
len(article_text)

8844

# Preprocessing

In [8]:
# Removing special characters and digits
formatted_article_text = re.sub('[^a-zA-Z]', ' ', article_text )
formatted_article_text = re.sub(r'\s+', ' ', formatted_article_text)

In [9]:
len(article_text)

8844

# Conversion de texte en phrases

In [10]:
sentence_list = nltk.sent_tokenize(article_text)

In [11]:
print (sentence_list)

['Natural language processing (NLP) is a subfield of linguistics, computer science, and artificial intelligence concerned with the interactions between computers and human language, in particular how to program computers to process and analyze large amounts of natural language data.', 'The goal is a computer capable of "understanding" the contents of documents, including the contextual nuances of the language within them.', 'The technology can then accurately extract information and insights contained in the documents as well as categorize and organize the documents themselves.', 'Challenges in natural language processing frequently involve speech recognition, natural-language understanding, and natural-language generation.', 'Natural language processing has its roots in the 1950s.', 'Already in 1950, Alan Turing published an article titled "Computing Machinery and Intelligence" which proposed what is now called the Turing test as a criterion of intelligence, though at the time that wa

# Trouver la fréquence d'occurrence pondérée

In [12]:
stopwords = nltk.corpus.stopwords.words('english')

word_frequencies = {}
for word in nltk.word_tokenize(formatted_article_text):
    if word not in stopwords:
        if word not in word_frequencies.keys():
            word_frequencies[word] = 1
        else:
            word_frequencies[word] += 1

In [13]:
maximum_frequncy = max(word_frequencies.values())

for word in word_frequencies.keys():
    word_frequencies[word] = (word_frequencies[word]/maximum_frequncy)

# Calcul des scores(poids) de phrase

In [14]:
sentence_scores = {}
for sent in sentence_list:
    for word in nltk.word_tokenize(sent.lower()):
        if word in word_frequencies.keys():
            if len(sent.split(' ')) < 30:
                if sent not in sentence_scores.keys():
                    sentence_scores[sent] = word_frequencies[word]
                else:
                    sentence_scores[sent] += word_frequencies[word]

# Obtenir le résumé

In [15]:
import heapq
summary_sentences = heapq.nlargest(7, sentence_scores, key=sentence_scores.get)

summary = ' '.join(summary_sentences)
print(summary)

Starting in the late 1980s, however, there was a revolution in natural language processing with the introduction of machine learning algorithms for language processing. In the 2010s, representation learning and deep neural network-style machine learning methods became widespread in natural language processing. That popularity was due partly to a flurry of results showing that such techniques can achieve state-of-the-art results in many natural language tasks, e.g., in language modeling and parsing. Up to the 1980s, most natural language processing systems were based on complex sets of hand-written rules. The cache language models upon which many speech recognition systems now rely are examples of such statistical models. Challenges in natural language processing frequently involve speech recognition, natural-language understanding, and natural-language generation. The following is a list of some of the most commonly researched tasks in natural language processing.


In [21]:
pip install -U scikit-learn

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 7.5/7.5 MB 4.5 MB/s eta 0:00:00
     ---------------------------------------- 40.1/40.1 MB 5.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [27]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
from nltk.util import LazyConcatenation, LazyMap

In [40]:
print("-" * 75)
test_sents = summary.split()
reference = '''
C'est au début des années 90 qu'Internet va connaître un formidable essor avec
l'arrivée du Web.
Le mot Internet vient de l’anglais Interconnected Networks.
Ces réseaux se communiquent entre eux grâce à des règles d’échanges de
données standard. Ces règles sont connues sous le nom de protocoles et
permettent de faire communiquer plusieurs ordinateurs et plusieurs réseaux entre
eux.
Le Web ou le World Wide Web (d’où l’acronyme WWW) permet de consulter
des sites web en utilisant le protocole HTTP (HyperText Transfert Protocol).
Internet met à la disposition de ses utilisateurs un grand nombre de ressources
(informations, logiciels, images, vidéos, etc.), il se présente comme une énorme
bibliothèque.
''' 

print("-" * 75)


def accuracy(reference, test):
	if len(reference) != len(test):
		raise ValueError("Lists must have the same length.")
	return sum(x == y for x, y in zip(reference, test)) / len(test)
def precision(reference, test):
	if not hasattr(reference, "intersection") or not hasattr(test, "intersection"):
	    raise TypeError("reference and test should be sets")
		
	if len(test) == 0:
		return None
	else:
		return len(reference.intersection(test)) / len(test)
		

def recall(reference, test):
	if not hasattr(reference, "intersection") or not hasattr(test, "intersection"):
		raise TypeError("reference and test should be sets")

	if len(reference) == 0:
		return None
	else:
		return len(reference.intersection(test)) / len(reference)
def f_measure(reference, test, alpha=0.5):
	p = precision(reference, test)
	r = recall(reference, test)
	if p is None or r is None:
		return None
	if p == 0 or r == 0:
		return 0
	return 1.0 / (alpha / p + (1 - alpha) / r)

def demo():
	print("-" * 75)
	reference = reference2.split()
	test = test_sents.split()
	print("Reference =", reference)
	print("Test    =", test)
	print("Accuracy:", accuracy(reference, test))

	print("-" * 75)
	reference_set = set(reference)
	test_set = set(test)
	print("Reference =", reference_set)
	print("Test =   ", test_set)
	print("Precision:", precision(reference_set, test_set))
	print("   Recall:", recall(reference_set, test_set))
	print("F-Measure:", f_measure(reference_set, test_set))
	print("-" * 75)

---------------------------------------------------------------------------
---------------------------------------------------------------------------


In [41]:
if __name__ == "__main__":
    demo()

---------------------------------------------------------------------------


NameError: name 'reference2' is not defined